In [1]:
import random
import time
import numpy as np
import torch

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_sequence(len_seq):
    x = np.random.randint(0, 9, len_seq)
    y = []
    y.append(x[0])
    for i in x[1:]:
        yi = i + x[0]
        if yi >= 10:
            yi = yi - 10
        y.append(yi)
    y = np.array(y)
    return x , y

In [3]:
def scaling_window(X, Y, seq_length):
    x = []
    y = []

    for i in range(len(X)-seq_length-1):
        _x = X[i:(i+seq_length)]
        _y = Y[i:(i+seq_length)]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)


x, y = generate_sequence(10000)

seq_length = 6
xw, yw = scaling_window(x, y, seq_length)

In [4]:
class RnnFlex(torch.nn.Module):

    def __init__(self, rnnClass):
        super().__init__()
        self.embed = torch.nn.Embedding(10, 32)
        self.hidden = rnnClass(32, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.hidden(embed)
        return self.linear(o)

In [5]:
X = torch.zeros((len(xw), seq_length), dtype=int)
Y = torch.zeros((len(xw), seq_length), dtype=int)

for i, seq in enumerate(xw):
    for t, num in enumerate(seq):
        X[i, t] = num
        Y[i, t] = yw[i, t]
print(X.shape, Y.shape)

torch.Size([9993, 6]) torch.Size([9993, 6])


In [6]:
dataset = torch.utils.data.TensorDataset(X, Y)

BATCH_SIZE = 32

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dl = torch.utils.data.DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, BATCH_SIZE, shuffle=True)

In [7]:

num_epochs = 20
learning_rate = 0.01

model = RnnFlex(torch.nn.RNN)

# criterion = torch.nn.MSELoss()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for x_in, y_in in train_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x_in).view(-1, 10)
        l = criterion(out, y_in)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    for x_in, y_in in test_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        out = model.forward(x_in).view(-1, 10)
        l = criterion(out, y_in)
        test_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )

Epoch: 0, loss: 7.0397, acc: 0.9958 | test loss: 0.0026, test acc: 1.0000 | 0.71 sec.
Epoch: 1, loss: 0.0768, acc: 1.0000 | test loss: 0.0140, test acc: 1.0000 | 0.65 sec.
Epoch: 2, loss: 0.0659, acc: 1.0000 | test loss: 0.0009, test acc: 1.0000 | 0.67 sec.
Epoch: 3, loss: 0.0632, acc: 1.0000 | test loss: 0.0007, test acc: 1.0000 | 0.65 sec.
Epoch: 4, loss: 0.0633, acc: 1.0000 | test loss: 0.0005, test acc: 1.0000 | 0.66 sec.
Epoch: 5, loss: 0.0620, acc: 1.0000 | test loss: 0.0005, test acc: 1.0000 | 0.64 sec.
Epoch: 6, loss: 0.0593, acc: 1.0000 | test loss: 0.0003, test acc: 1.0000 | 0.65 sec.
Epoch: 7, loss: 0.0622, acc: 1.0000 | test loss: 0.0003, test acc: 1.0000 | 0.65 sec.
Epoch: 8, loss: 0.0648, acc: 1.0000 | test loss: 0.0003, test acc: 1.0000 | 0.65 sec.
Epoch: 9, loss: 0.0661, acc: 1.0000 | test loss: 0.0003, test acc: 1.0000 | 0.64 sec.
Epoch: 10, loss: 0.0647, acc: 1.0000 | test loss: 0.0003, test acc: 1.0000 | 0.65 sec.
Epoch: 11, loss: 0.0653, acc: 1.0000 | test loss: 0.0

In [8]:
def predict():
    x_in, y_in = random.choice(test_dataset)
    model.eval()
    outputs = model.forward(x_in).view(-1, 10)
    print('x: {},\ny: {},\ny_pred: {}'.format(x_in, y_in, outputs.argmax(dim=1)))

predict()

x: tensor([5, 8, 5, 4, 8, 6]),
y: tensor([2, 5, 2, 1, 5, 3]),
y_pred: tensor([2, 5, 2, 1, 5, 3])
